# this notebook is used to build train and test dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%run /Users/yaoyao/Documents/datascience/toolbox/toolbox1_dataview.ipynb
%matplotlib inline

## load libs and readin data

In [2]:
with open('train.csv') as f:
    raw_train = pd.read_csv(f)
with open('test.csv') as f:
    raw_test = pd.read_csv(f)

## raw clean the train

In [3]:
def get_devive(train):
    # 1. we're predicting the attempts of last assessment for every installid
    # so first we need to remove other activities after the last assessment
    idlist = train['installation_id'].unique()
    greplast = train.drop_duplicates(subset = 'installation_id',keep='last').index
    dictionary = dict(zip(idlist, greplast))
    del idlist
    del greplast
    return dictionary

def train_clean(raw_train):
    # 1.get installation_id with only one game session
    # all records doing assessments
    astrain = raw_train[raw_train['type'] == 'Assessment'].loc[:,['installation_id','game_session']]
    # all insid ever done assessment
    inid2keep = astrain.groupby('installation_id').count().index
    train = raw_train[raw_train['installation_id'].isin(inid2keep)]
    
    #2, remove last events are not assessment
    # loop through devices to find index need to be dropped.
    dictionary = get_devive(train)
    train_cp = train.loc[:,['game_session','type','installation_id']]
    train_unique = train_cp.drop_duplicates(subset='game_session',keep='last')
    first = 0
    for device in dictionary:
        first += 1
        cut = train_unique[(train_unique['installation_id'] == device)]
        for idx,row in cut.iloc[::-1].iterrows():
            if row['type'] == 'Assessment':
                if first == 1:
                    rmidx = train_cp[(train_cp.index > idx) & (train_cp.index <= dictionary[device])].index
                else:
                    rmidx = rmidx.append(train_cp[(train_cp.index > idx) & (train_cp.index <= dictionary[device])].index)
                break
        if first % 100 == 0:
            print("processed",first,"ids")
    # drop the events
    print("dropping events, now we have valid training set:train")
    ptrain = train.drop(index=rmidx)
    
    #3,output testformatset, and raw_y
    # to further split last assessment for each installation_id
    train_cp = ptrain.loc[:,['game_session','type','installation_id']]
    unique = train_cp.drop_duplicates(subset='game_session',keep='last')
    lastassessment = unique.drop_duplicates(subset='installation_id',keep='last')
    fullresults = ptrain[ptrain['game_session'].isin(lastassessment.game_session)]
    firstlast = fullresults.loc[:,['game_session','installation_id']].drop_duplicates(subset='installation_id',keep='first')
    dropindex = fullresults[~fullresults.index.isin(firstlast.index)].index
    opt1 = ptrain[~ptrain.index.isin(dropindex)]
    # return, 1:all full last assessment for each installation_id
    #         2:2nd- end events removed for the last assessment for each installation id, should be same withtest
    return fullresults, opt1

In [4]:
raw_y,train_ready = train_clean(raw_train)

processed 100 ids
processed 200 ids
processed 300 ids
processed 400 ids
processed 500 ids
processed 600 ids
processed 700 ids
processed 800 ids
processed 900 ids
processed 1000 ids
processed 1100 ids
processed 1200 ids
processed 1300 ids
processed 1400 ids
processed 1500 ids
processed 1600 ids
processed 1700 ids
processed 1800 ids
processed 1900 ids
processed 2000 ids
processed 2100 ids
processed 2200 ids
processed 2300 ids
processed 2400 ids
processed 2500 ids
processed 2600 ids
processed 2700 ids
processed 2800 ids
processed 2900 ids
processed 3000 ids
processed 3100 ids
processed 3200 ids
processed 3300 ids
processed 3400 ids
processed 3500 ids
processed 3600 ids
processed 3700 ids
processed 3800 ids
processed 3900 ids
processed 4000 ids
processed 4100 ids
processed 4200 ids
dropping events, now we have valid training set:train


In [7]:
raw_y.to_csv('raw_y.csv',index=False)

In [8]:
train_ready.to_csv('train_ready.csv',index=False)